In [ ]:
import fasttext
import unidecode
import string

import pandas as pd

from sklearn.model_selection import train_test_split

# Importação dos dados

In [ ]:
df = pd.read_csv('/kaggle/input/imdb-ptbr/imdb-reviews-pt-br.csv')
df.head()

![](http://)Para modelar utilizando a biblioteca fasttext a variável resposta precisa estar no formato \_\_label\_\_(valor), exemplo: \_\_label\_\_positi[](http://)vo.

In [ ]:
# Colocando a coluna id como index do dataset:
df.set_index('id', inplace=True) 


# Transformando o target para utilizar o fasttext:
df['target']=['__label__'+ s for s in df['sentiment']]

In [ ]:
# Verificando a distribuição da variável resposta
df['target'].value_counts()

# Pré-processamento

## Limpeza do texto

Vamos remover acentuação e pontuação dos textos, além de colocar tudo em caixa baixa.

In [ ]:
# Colocando todas as palavras em caixa baixa:
df['text_pt'] = df['text_pt'].str.lower()
# Removendo acentuação:
df['text_pt'] = df['text_pt'].apply(lambda text: unidecode.unidecode(text))
# Removendo pontuação:
df['text_pt'] = df['text_pt'].str.replace('[{}]'.format(string.punctuation), '')

O resultado fica assim:

In [ ]:
df['text_pt'].head()

# Modelagem

Primeiramente vamos separar a base em treino e teste:

In [ ]:
# Utilizando a proporção de 70/30 para bases de treino e teste, respectivamente:

X = df['text_pt']
Y = df['target']

X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    Y,
                                                    test_size = 0.3,
                                                    random_state = 42)

Outra necessidade da biblioteca fasttext é que a entrada do modelo esteja em um arquivo, por isso, criar novos dataframes e salvar em arquivos separados.

In [ ]:
# Reunindo novamente as bases de treino e teste:

data_train = {'target': Y_train, 'text': X_train}
df_train = pd.DataFrame.from_dict(data_train)
data_test = {'target': Y_test, 'text': X_test}
df_test = pd.DataFrame.from_dict(data_test)

In [ ]:
# Salvando as bases de treino e teste em arquivos separados:

df_train.to_csv('df_train.csv', sep='\t', index = False, header = False)
df_test.to_csv('df_test.csv', sep='\t', index = False, header = False)

Treinando o modelo:

In [ ]:
model = fasttext.train_supervised(input='df_train.csv')

No teste do modelo, as medidas que a função exibe são a quantidade de registros da base de teste, a precisão e o revocação, respectivamente.

In [ ]:
model.test('df_test.csv')

**Conseguimos valores na casa dos 88% de precisão e revocação.**